# Test access to minio buckets without spark

This test comes from https://github.com/marcoverl/training/blob/master/CCR/S3-with-boto3.ipynb

In [1]:
#!pip install liboidcagent requests xmltodict pandas boto3

In [2]:
!eval `oidc-keychain` > /dev/null && oidc-token dodas --time=3600 > /tmp/token
with open('/tmp/token') as f:
    token = f.readlines()[0].split("\n")[0]

In [3]:
import requests
import xmltodict
r = requests.post("https://minio.cloud.infn.it",
                  data={
                      'Action':
                      "AssumeRoleWithWebIdentity",
                      'Version': "2011-06-15",
                      'WebIdentityToken': token,
                      'DurationSeconds': 9000
                  },
                  verify=True)

tree = xmltodict.parse(r.content)

credentials = dict(tree['AssumeRoleWithWebIdentityResponse']
                    ['AssumeRoleWithWebIdentityResult']['Credentials'])

In [4]:
import pandas as pd
import boto3
s3 = boto3.client('s3',endpoint_url="https://minio.cloud.infn.it",aws_access_key_id=credentials["AccessKeyId"],
                  aws_secret_access_key=credentials["SecretAccessKey"],
                  aws_session_token=credentials["SessionToken"],
                  config=boto3.session.Config(signature_version='s3v4'),
                 verify=True)

In [5]:
read_file = s3.get_object(Bucket='scratch', Key='verlato/dpc-covid19-ita-regioni.csv')
df = pd.read_csv(read_file['Body'],sep=',')
df

,data,stato,codice_regione,denominazione_regione,lat,long,ricoverati_con_sintomi,terapia_intensiva,totale_ospedalizzati,isolamento_domiciliare,...,deceduti,casi_da_sospetto_diagnostico,casi_da_screening,totale_casi,tamponi,casi_testati,note,ingressi_terapia_intensiva,note_test,note_casi
0,2020-02-24T18:00:00,ITA,13,Abruzzo,42.351222,13.398438,0,0,0,0,...,0,NaN,NaN,0,5,NaN,NaN,NaN,NaN,NaN
1,2020-02-24T18:00:00,ITA,17,Basilicata,40.639471,15.805148,0,0,0,0,...,0,NaN,NaN,0,0,NaN,NaN,NaN,NaN,NaN
2,2020-02-24T18:00:00,ITA,18,Calabria,38.905976,16.594402,0,0,0,0,...,0,NaN,NaN,0,1,NaN,NaN,NaN,NaN,NaN
3,2020-02-24T18:00:00,ITA,15,Campania,40.839566,14.250850,0,0,0,0,...,0,NaN,NaN,0,10,NaN,NaN,NaN,NaN,NaN
4,2020-02-24T18:00:00,ITA,8,Emilia-Romagna,44.494367,11.341721,10,2,12,6,...,0,NaN,NaN,18,148,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6358,2020-12-22T17:00:00,ITA,19,Sicilia,38.115697,13.362357,1059,176,1235,32257,...,2203,NaN,NaN,86092,1155288,773208.0,NaN,7.0,NaN,NaN
6359,2020-12-22T17:00:00,ITA,9,Toscana,43.769231,11.255889,941,175,1116,10721,...,3495,NaN,NaN,116544,1806133,1035769.0,NaN,3.0,Positivi diagnosticati solo con test antigenic...,NaN
6360,2020-12-22T17:00:00,ITA,10,Umbria,43.106758,12.388247,244,44,288,3441,...,578,NaN,NaN,27396,481344,245205.0,NaN,5.0,NaN,NaN
6361,2020-12-22T17:00:00,ITA,2,Valle d'Aosta,45.737503,7.320149,90,5,95,361,...,368,NaN,NaN,7073,60537,36564.0,NaN,1.0,NaN,NaN


In [6]:
read_file = s3.get_object(Bucket='legger', Key='NLPInput/message_example.csv')
df = pd.read_csv(read_file['Body'],sep=',')
df

,error_category,error_message
0,PERMISSION_DENIED,[gfalt_copy_file][perform_copy][gfal_http_copy...
1,PERMISSION_DENIED,[gfalt_copy_file][perform_copy][gfal_http_copy...
2,COMMUNICATION_ERROR_ON_SEND,DESTINATION OVERWRITE srm-ifce err: Communicat...
3,INVALID_REQUEST_DESCRIPTOR,DESTINATION MAKE_PARENT srm-ifce err: Invalid ...
4,CONNECTION_TIMED_OUT,DESTINATION SRM_PUT_TURL srm-ifce err: Connect...
...,...,...
711823,COMMUNICATION_ERROR_ON_SEND,DESTINATION OVERWRITE srm-ifce err: Communicat...
711824,COMMUNICATION_ERROR_ON_SEND,DESTINATION OVERWRITE srm-ifce err: Communicat...
711825,COMMUNICATION_ERROR_ON_SEND,DESTINATION OVERWRITE srm-ifce err: Communicat...
711826,COMMUNICATION_ERROR_ON_SEND,DESTINATION OVERWRITE srm-ifce err: Communicat...


In [7]:
read_file = s3.get_object(Bucket='legger', Key='MLCourseInput/Higgs1M.csv')
df = pd.read_csv(read_file['Body'],sep=',')
df

,Unnamed: 0,label,lepton_pT,lepton_eta,lepton_phi,missing_energy_magnitude,missing_energy_phi,jet1_pt,jet1_eta,jet1_phi,...,je4_eta,jet4_phi,jet4_b-tag,m_jj,m_jjj,m_lv,m_jlv,m_bb,m_wbb,m_wwbb
0,0,1.0,0.869293,-0.635082,0.225690,0.327470,-0.689993,0.754202,-0.248573,-1.092064,...,-0.010455,-0.045767,3.101961,1.353760,0.979563,0.978076,0.920005,0.721657,0.988751,0.876678
1,1,1.0,0.907542,0.329147,0.359412,1.497970,-0.313010,1.095531,-0.557525,-1.588230,...,-1.138930,-0.000819,0.000000,0.302220,0.833048,0.985700,0.978098,0.779732,0.992356,0.798343
2,2,1.0,0.798835,1.470639,-1.635975,0.453773,0.425629,1.104875,1.282322,1.381664,...,1.128848,0.900461,0.000000,0.909753,1.108330,0.985692,0.951331,0.803252,0.865924,0.780118
3,3,0.0,1.344385,-0.876626,0.935913,1.992050,0.882454,1.786066,-1.646778,-0.942383,...,-0.678379,-1.360356,0.000000,0.946652,1.028704,0.998656,0.728281,0.869200,1.026736,0.957904
4,4,1.0,1.105009,0.321356,1.522401,0.882808,-1.205349,0.681466,-1.070464,-0.921871,...,-0.373566,0.113041,0.000000,0.755856,1.361057,0.986610,0.838085,1.133295,0.872245,0.808487
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
999994,999994,0.0,1.289299,-1.108431,0.970869,0.418467,-0.236384,1.439424,1.415013,0.384343,...,-0.196174,-0.649514,3.101961,3.715858,2.295703,0.990350,0.826123,0.602524,1.549889,1.489945
999995,999995,1.0,0.514256,0.441154,0.641281,0.376795,0.840085,0.812464,-0.038644,-0.329241,...,0.032852,1.471465,0.000000,0.813464,0.723074,0.984661,0.734838,0.226974,0.817835,0.750583
999996,999996,1.0,1.586689,0.798600,-1.038389,0.186234,-1.465364,1.328488,0.356457,0.846692,...,0.155277,-0.170067,3.101961,2.318721,1.348919,0.997061,1.345282,0.984395,1.167846,1.042671
999997,999997,0.0,1.833751,0.194740,1.489664,0.453500,0.077691,1.112570,1.208055,0.155386,...,0.112803,0.599699,0.000000,1.033291,0.878243,1.207249,0.980557,1.977671,1.186537,1.025073
